### Animated map which displays the protests locations in sequence
With:
- ##### <u> SQLDATE </u>
- ##### <u> ActionGeo_Lat </u>
- ##### <u> ActionGeo_Long </u>

In [48]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import os
from IPython.core.display import display, HTML

DATA_PATH = "data/"
LEAFLET_PATH = 'Leaflet.MovingMarker-master\\animated_map\\'

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'C:\\Users\\antho\\Downloads\\ADAproj-02b470a29ee4.json'
bigquery_client = bigquery.Client()

In [49]:
#Fetching data from GDELT
## BIG DATA, 3 000 000 rows

if os.path.isfile(DATA_PATH + 'protests_location.csv') :
    #If we already saved the data, don't run the query again, just get the data from the file saved previously
    protests_df_location = pd.read_csv(DATA_PATH + 'protests_location.csv')
else:
    #query to get the date and the location of the protest events
    #Remark: '14%' filters the protest events because they all start by '14'
    query_protests_location = bigquery_client.query(
        """SELECT SQLDATE, ActionGeo_Lat, ActionGeo_Long FROM `gdelt-bq.gdeltv2.events` 
        WHERE EventCode LIKE '14%' """)
    protests_df_location = query_protests_location.result().to_dataframe()
    # Write down a csv file
    protests_df_location.to_csv(DATA_PATH + 'protests_location.csv', index=False)
protests_df_location = protests_df_location.dropna()
protests_df_location.count()

SQLDATE           3756896
ActionGeo_Lat     3756896
ActionGeo_Long    3756896
dtype: int64

In [50]:
#Extracting ActionGeo_Lat and ActionGeo_Long
protests_without_duplicated_values = pd.DataFrame()
protests_without_duplicated_values['ActionGeo_Lat'] = protests_df_location['ActionGeo_Lat']
protests_without_duplicated_values['ActionGeo_Long'] = protests_df_location['ActionGeo_Long']

#Removing duplicated values 
#The values are not really duplicated, they took place on different day
#Plus, the localization of national protest is in the center of the country 
#We thus have a duplicated localizations when we have several national protests in the same country
protests_without_duplicated_values=protests_without_duplicated_values.drop_duplicates(subset=['ActionGeo_Long', 'ActionGeo_Lat'], keep=False)

In [51]:
protests_without_duplicated_values["coord_for_js"] = '[' + protests_without_duplicated_values['ActionGeo_Lat'].astype(str) + ',' + protests_without_duplicated_values['ActionGeo_Long'].astype(str) + '],'
protests_coordinates1 = ""
protests_coordinates2 = ""
idx = 0
for value in protests_without_duplicated_values["coord_for_js"]:
    idx = idx + 1
    if idx < len(protests_without_duplicated_values["coord_for_js"])/2:
        protests_coordinates1 = protests_coordinates1 + value
    else:
        protests_coordinates2 = protests_coordinates2 + value

string_addstation1=""
string_addstation2=""
j = 0;
for i in range(1, len(protests_without_duplicated_values["coord_for_js"])-1):
    if i < len(protests_without_duplicated_values["coord_for_js"])/2:
        string_addstation1 = string_addstation1 + "marker1.addStation(" + str(i) + ", 500);"
    else:
        string_addstation2 = string_addstation2 + "marker2.addStation(" + str(j) + ", 500);"
        j=j+1


In [52]:
readFile = open(LEAFLET_PATH + "script.js")

lines = readFile.readlines()
lines = lines[:-1]
readFile.close()

lines[20] = protests_coordinates1[:-1]
lines.insert(21, '\n')
lines[26] = protests_coordinates2[:-1]
lines.insert(27, '\n')
lines[40] = string_addstation1
lines.insert(41, '\n')
lines.append(string_addstation2)

In [53]:
w = open(LEAFLET_PATH + "script.js",'w')

w.writelines([item for item in lines])

w.close()

In [54]:
display(HTML("<h1><a href='"+LEAFLET_PATH+"index.html' target='_blank'>Animated map that displays protests location in sequence (Ctrl+Click)</a></h1>"))